# Gen AI Intensive Course Capstone 2025Q1
## Monopoly rules assistant using RAG

## Project Overview
#### In this example, we’ll build a Board Game Rules Assistant that lets users ask questions about the rules of a board game (for example, from a PDF rulebook).

**Goal**: Create an interactive Jupyter Notebook that can read a board game rule document (e.g., in PDF form), index its contents with embeddings, and answer user queries in a structured JSON format using few-shot prompting in a RAG framework.

Generative AI capabilities demonstrated:

**Retrieval Augmented Generation (RAG)**: The notebook ingests the document, splits it into chunks, computes embeddings, and stores these in a vector index . When a user asks a question, the notebook retrieves the most relevant text passages for context.

**Few-shot Prompting**: The prompt fed into the language model includes a couple of examples (a few-shot setup), instructing the system on how to answer questions based solely on the retrieved context. We guide the large language model with a few example question–answer pairs, which helps steer the generation in a specific style.

**Structured Output/JSON Mode**: The language model is explicitly instructed to output its answer in JSON format (e.g., with keys "answer", and "sources"), ensuring that its responses are controlled and easy to postprocess.

Additional optional capabilities you might consider include using document understanding to parse PDFs robustly, incorporating a vector store (Embeddings & Vector search), or even MLOps practices to record experiment metrics.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/monopoly-rules/Monopoly Rules.pdf


## A: Setup and Library installation
We install ChromaDB, Gemini API, and pymupdf

In [2]:
!pip install -q sentence-transformers chromadb google-generativeai pymupdf
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import fitz  
import os
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 8.1 MB/s eta 0:00:

2025-04-20 15:53:45.294746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745164425.572286      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745164425.664303      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Setting up API Key and selecting a model

In [3]:
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
api_key = secrets.get_secret("GOOGLE_API_KEY")
genai.configure(api_key=api_key)
# Gemini model
model_gemini = genai.GenerativeModel('gemini-2.0-flash')

## B: Document Understanding: Extracting Text from a PDF
We have a PDF file for our board game rules. Extract the text and split it into manageable chunks.
A popular board game is 'Monopoly.' I've decided to use its rules here.
Found here [https://officialgamerules.org/wp-content/uploads/2025/02/00009.pdf].

In [4]:
#Extracting
def load_pdf_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_path = "/kaggle/input/monopoly-rules/Monopoly Rules.pdf"
full_text = load_pdf_text(pdf_path)

#Chunking
def chunk_text(text, max_tokens=300):
    import nltk
    nltk.download('punkt')
    from nltk.tokenize import sent_tokenize

    sentences = sent_tokenize(text)
    chunks, current_chunk = [], []

    current_length = 0
    for sentence in sentences:
        tokens = sentence.split()
        if current_length + len(tokens) > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(sentence)
        current_length += len(tokens)

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

chunks = chunk_text(full_text)

print(f"Extracted {len(chunks)} chunks from the document.")

Extracted 10 chunks from the document.


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## C: Creating Embeddings and building a vector store
For each text chunk, we generate an embedding. Then store all embeddings in ChromaDB for efficient retrieval.

In [5]:
#Create embeddings with SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks).tolist()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
#Create Vector Store using ChromaDB
from chromadb.utils import embedding_functions
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(
    name="monopoly_rag",
    embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
)
existing_ids = set(collection.get()["ids"])
for i, chunk in enumerate(chunks):
    doc_id = f"chunk_{i}"
    if doc_id not in existing_ids:
        collection.add(documents=[chunk], embeddings=[embeddings[i]], ids=[doc_id])

## D: Retrieving relevant context for a query, few-shot prompting and structured output generation (JSON)
**We build the RAG pipeline.**

Define a function that, given a user query, computes its embedding, retrieves the top-k relevant chunks from chromaDB, and aggregates them as context. Then, build a prompt that includes a few-shot example and instructs the model to use only the retrieved context. The response should be in a structured JSON format.

In [7]:
def rag_qa_pipeline(user_query, k=4):
    query_embedding = model.encode([user_query])[0]
    results = collection.query(query_embeddings=[query_embedding], n_results=k)

    context_chunks = [doc for doc in results['documents'][0]]
    context = "\n\n".join(context_chunks)

    # Few-shot prompt template with examples
    few_shot_examples = """
    Example 1:
    Question: How many players can play at most?
    Context: [Relevant text from the rules]
    Answer: {"answer": "The game supports up to 4 players.", "sources": ["Section 2.1", "Page 5"]}
    
    Example 2:
    Question: What action is taken when a player cannot move?
    Context: [Relevant text from the rules]
    Answer: {"answer": "The player loses their turn.", "sources": ["Section 3.3", "Page 10"]}
    
    Now, based on the following context extracted from the board game rules, answer the question in the same JSON format.
    """
    prompt = f"""You are an assistant that answers questions based on Monopoly rules.
    Based only on the context provided below, answer the given question using the same format as the examples.:

    Question: {user_query}
    Context: {context}
    Answer:

{few_shot_examples}"""
    
    response = model_gemini.generate_content(prompt)
    return response.text

#### See if it works.

In [8]:
# Test question
test_question = "Can I collect rent while in jail?"
answer = rag_qa_pipeline(test_question)
print(f"\nQuestion: {test_question}\nAnswer:\n{answer}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Question: Can I collect rent while in jail?
Answer:
Question: Can you buy property while in jail?
Context: You then get out of Jail and immediately move forward the number 
of spaces shown by your throw. Even though you are in Jail, you may buy and sell property, buy and 
sell houses and hotels and collect rents.
Answer: {"answer": "Even though you are in Jail, you may buy and sell property.", "sources": ["Page 1"]}


### Try it further-
#### Uncomment the following block to ask more questions. Examples-
- What happens when you land on Free Parking?
- Do I have to land on a property to buy it?
- Can I build houses unevenly?
- What happens if I can’t pay rent or debt?

In [9]:
# while True:
#     user_input = input("\n Welcome to the Monopoly RAG Bot. Ask a question about the rules of Monopoly (or type 'exit' to quit):\n> ")
#     if user_input.lower() in ["exit", "quit", "q", "buy"]:
#         print("Goodbye! 👋")
#         break
#     answer = rag_qa_pipeline(user_input)
#     print(f"\nAnswer:\n{answer}")

## Discussion and Next Steps
This project brings together several modern Gen AI techniques:

- Data ingestion & Document Understanding: Extract and clean unstructured board game rules.
- Embeddings & Vector Search (RAG): Efficiently retrieve the most relevant segments of a document.
- Few-shot Prompting & Structured Output: Guide the language model to produce controlled, JSON-structured responses.

Possible extensions:
    
- Incorporate additional modalities (e.g., image understanding if the board game has diagrammatic instructions)    
- Add a user interface (using Streamlit or Voila) for interactive Q&A sessions.
- Log prompts and model responses for MLOps monitoring and continuous improvement.
    
This design can be adapted for other use cases such as summarizing the latest news, creating a customer support agent, or even understanding video transcripts. The blend of RAG, few-shot prompting, and structured generation offers a robust template for many real-world applications.

**Thank You.**